In [6]:
import os
import ee
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import math
import skgstat as skg

In [7]:
# Create a project called "ee-sentinel-2temporal-nir"
# Google Account is jiawei.gao.polimi@gmail.com
ee.Authenticate(force = False)
ee.Initialize(project='ee-sentinel-2temporal-nir')

In [8]:
cwd = "c:\\Users\\m1865\\Desktop\\DISC"
cwd_Images_Raw = cwd + "\\Sentinel-2 Images Raw"
cwd_Images_Processed = cwd + "\\Sentinel-2 Images Processed"

In [9]:
date_Start = "2023-01-01"
date_End = "2023-12-31"
name_Site = "CD-Ygb"
roi_Distance = 100

In [10]:
# Read shapefile of ROI
gdf_ROI = gpd.read_file(cwd_Images_Processed + "\\" + name_Site + "\\" + str(roi_Distance) + "m.shp")
# Get its crs
EPSG_ROI = gdf_ROI.crs
EPSG_ROI

<Projected CRS: EPSG:32634>
Name: WGS 84 / UTM zone 34N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 18°E and 24°E, northern hemisphere between equator and 84°N, onshore and offshore. Albania. Belarus. Bosnia and Herzegovina. Bulgaria. Central African Republic. Chad. Croatia. Democratic Republic of the Congo (Zaire). Estonia. Finland. Greece. Hungary. Italy. Kosovo. Latvia. Libya. Lithuania. Montenegro. North Macedonia. Norway, including Svalbard and Bjornoys. Poland. Romania. Russian Federation. Serbia. Slovakia. Sudan. Sweden. Ukraine.
- bounds: (18.0, 0.0, 24.0, 84.0)
Coordinate Operation:
- name: UTM zone 34N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [11]:
# Create a GEE ROI in EPSG:4326
gdf_ROI_4326 = gdf_ROI.to_crs("EPSG:4326")
bounds = gdf_ROI_4326.bounds.values.reshape(-1)
roi = ee.Geometry.Rectangle([bounds[0],bounds[1],bounds[2],bounds[3]])

In [12]:
# Get the centroid of ROI
centroid = gdf_ROI.centroid
centroid_x = centroid.get_coordinates().iloc[0,0]
centroid_y = centroid.get_coordinates().iloc[0,1]
# Now we get all coordinates of pixels in our ROI
# Get all the coordinates of pixel inside ROI
arr_x = np.arange(centroid_x - roi_Distance / 2, centroid_x + roi_Distance / 2 + 10, 10)
arr_y = np.arange(centroid_y - roi_Distance / 2, centroid_y + roi_Distance / 2 + 10, 10)
arr_xy = np.empty(shape = (len(arr_x)*len(arr_y),2), dtype = int)
z = 0
for j in range(len(arr_y)):
    for i in range(len(arr_x)):
        arr_xy[z,0] = arr_x[i]
        arr_xy[z,1] = arr_y[j]
        z = z + 1
# Create a new geodataframe
df_Points = pd.DataFrame({
    'x': arr_xy[:,0],
    'y': arr_xy[:,1]
})
gdf_Points = gpd.GeoDataFrame(df_Points, geometry = gpd.points_from_xy(df_Points['x'], df_Points['y'], crs=EPSG_ROI))
gdf_Points['x'] = gdf_Points.geometry.x
gdf_Points['y'] = gdf_Points.geometry.y
gdf_Points

,x,y,geometry
0,889895.0,90145.0,POINT (889895 90145)
1,889905.0,90145.0,POINT (889905 90145)
2,889915.0,90145.0,POINT (889915 90145)
3,889925.0,90145.0,POINT (889925 90145)
4,889935.0,90145.0,POINT (889935 90145)
...,...,...,...
116,889955.0,90245.0,POINT (889955 90245)
117,889965.0,90245.0,POINT (889965 90245)
118,889975.0,90245.0,POINT (889975 90245)
119,889985.0,90245.0,POINT (889985 90245)


# Read Sentinel-2 L1C Image Collection

In [13]:
dataset_L1C = (
    ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
    .filterBounds(roi)
    .filterDate(date_Start,date_End)
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))
    .sort("system:time_start")
)
imageList_L1C = dataset_L1C.toList(dataset_L1C.size())
"There are " + str(dataset_L1C.size().getInfo()) + " L1C images available!"

'There are 70 L1C images available!'

In [14]:
idList_L1C = []
for i in range(dataset_L1C.size().getInfo()):
    temp_ID = dataset_L1C.getInfo()['features'][i]['id']
    temp_ID = temp_ID.split('/')[-1]
    idList_L1C.append(temp_ID)

In [15]:
dataset_L2A = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(roi)
    .filterDate(date_Start,date_End)
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))
    .sort("system:time_start")
)
imageList_L2A = dataset_L2A.toList(dataset_L2A.size())
"There are " + str(dataset_L2A.size().getInfo()) + " L2A images available!"

'There are 74 L2A images available!'

In [16]:
idList_L2A = []
for i in range(dataset_L2A.size().getInfo()):
    temp_ID = dataset_L2A.getInfo()['features'][i]['id']
    temp_ID = temp_ID.split('/')[-1]
    idList_L2A.append(temp_ID)

In [17]:
# Filter out the images that don't appear in both ImageCollections
idList_L1C_Redun = []
imageList_L1C_NoRedun = ee.List([])
for i in range(len(idList_L1C)):
    if idList_L1C[i] not in idList_L2A:
        idList_L1C_Redun.append(idList_L1C[i])
    else:
        temp_Image = ee.Image(imageList_L1C.get(i))
        imageList_L1C_NoRedun = imageList_L1C_NoRedun.add(temp_Image)
redunList_L2A = []
imageList_L2A_NoRedun = ee.List([])
for i in range(len(idList_L2A)):
    if idList_L2A[i] not in idList_L1C:
        redunList_L2A.append(idList_L2A[i])
    else:
        temp_Image = ee.Image(imageList_L2A.get(i))
        imageList_L2A_NoRedun = imageList_L2A_NoRedun.add(temp_Image)
redunList_L2A

['20230304T082831_20230304T084529_T34NHG',
 '20230304T082831_20230304T084529_T35NKA',
 '20230304T082831_20230304T084529_T34NHF',
 '20230314T082721_20230314T084302_T34NHG',
 '20230324T082611_20230324T083858_T35NKA',
 '20230811T082611_20230811T084737_T34NHG',
 '20230811T082611_20230811T084737_T35NKA',
 '20230811T082611_20230811T084737_T34NHF',
 '20231209T083331_20231209T084928_T34NHF']

In [18]:
if imageList_L1C_NoRedun.size().getInfo() != imageList_L2A_NoRedun.size().getInfo():
    raise SystemExit("Stop right there!")
else:
    num_Images = imageList_L1C_NoRedun.size().getInfo()

In [19]:
imageList_L1C_NoRedun.getInfo()

[{'type': 'Image',
  'bands': [{'id': 'B1',
    'data_type': {'type': 'PixelType',
     'precision': 'int',
     'min': 0,
     'max': 65535},
    'dimensions': [1830, 1830],
    'crs': 'EPSG:32634',
    'crs_transform': [60, 0, 799980, 0, -60, 200040]},
   {'id': 'B2',
    'data_type': {'type': 'PixelType',
     'precision': 'int',
     'min': 0,
     'max': 65535},
    'dimensions': [10980, 10980],
    'crs': 'EPSG:32634',
    'crs_transform': [10, 0, 799980, 0, -10, 200040]},
   {'id': 'B3',
    'data_type': {'type': 'PixelType',
     'precision': 'int',
     'min': 0,
     'max': 65535},
    'dimensions': [10980, 10980],
    'crs': 'EPSG:32634',
    'crs_transform': [10, 0, 799980, 0, -10, 200040]},
   {'id': 'B4',
    'data_type': {'type': 'PixelType',
     'precision': 'int',
     'min': 0,
     'max': 65535},
    'dimensions': [10980, 10980],
    'crs': 'EPSG:32634',
    'crs_transform': [10, 0, 799980, 0, -10, 200040]},
   {'id': 'B5',
    'data_type': {'type': 'PixelType',
   

In [20]:
# Save date!
list_Date = []
for i in range(num_Images):
    # L1C
    image_L1C = ee.Image(imageList_L1C_NoRedun.get(i))
    temp_Date = image_L1C.getInfo()['id'].split('/')[-1][0:8]
    list_Date.append(temp_Date)

In [46]:
# Now start the real work
list_CV = []
list_Sill = []
for i in range(num_Images):
    # L1C
    image_L1C = ee.Image(imageList_L1C_NoRedun.get(i))
    L1C_info = image_L1C.getInfo()["properties"]
    L1C_solar_z = L1C_info['MEAN_SOLAR_ZENITH_ANGLE']
    L1C_offset_B8 = L1C_info['RADIO_ADD_OFFSET_B8']
    L1C_solar_B8 = L1C_info['SOLAR_IRRADIANCE_B8']
    L1C_u_B8 = L1C_info['REFLECTANCE_CONVERSION_CORRECTION']
    L1C_B8_Ref = image_L1C.select(7).divide(10000)
    # print('L1C')
    # L2A
    image_L2A = ee.Image(imageList_L2A_NoRedun.get(i))
    ndvi = image_L2A.normalizedDifference(['B8', 'B4']).rename('NDVI')
    # print('L2A')
    # L1C Radiance
    list_L1C_B8_Rad = []
    for j in range(gdf_Points.shape[0]):
        point = ee.Geometry.Point([gdf_Points.loc[j,'x'],gdf_Points.loc[j,'y']], proj='EPSG:32634')
        L1C_B8_Ref_repo = L1C_B8_Ref.reproject(crs = 'EPSG:32634')
        value = L1C_B8_Ref_repo.reduceRegion(
            reducer = ee.Reducer.mean(),
            geometry = point,
            scale = 10,
            crs = 'EPSG:32634'
        )
        pixelValues = value.getInfo()['B8']
        if pixelValues:
            # radiance = reflectance * cos(radians(SunZenithAngle)) * solarIrradiance * U / pi
            pixelValues_toRad = pixelValues * math.cos(math.radians(L1C_solar_z)) * L1C_solar_B8 * L1C_u_B8 / math.pi
        else:
            temp_Check = False
            break
        list_L1C_B8_Rad.append(pixelValues_toRad)
    if temp_Check:
        # L2A NDVI
        list_L2A_NDVI = []
        for j in range(gdf_Points.shape[0]):
            point = ee.Geometry.Point([gdf_Points.loc[j,'x'],gdf_Points.loc[j,'y']], proj='EPSG:32634')
            ndvi_repo = ndvi.reproject(crs = 'EPSG:32634')
            value = ndvi_repo.reduceRegion(
                reducer = ee.Reducer.mean(),
                geometry = point,
                scale = 10,
            )
            pixelValues = value.getInfo()['NDVI']
            if pixelValues:
                list_L2A_NDVI.append(pixelValues)
            else:
                temp_Check = False
                break
        if temp_Check:
            # NIRv Rad
            Nirv_Rad = np.multiply(list_L1C_B8_Rad,list_L2A_NDVI)
            # Dataframe
            gdf_Points['NIRv Rad'] = Nirv_Rad
            # CV
            CV = gdf_Points['NIRv Rad'].std() / gdf_Points['NIRv Rad'].mean()
            list_CV.append(CV)
            # Sill
            band = gdf_Points['NIRv Rad'].to_numpy().reshape(-1)
            diagonal = roi_Distance * (2 ** 0.5) / 2
            bin_Custom = np.arange(10,diagonal,10)
            V_Default = skg.Variogram(arr_xy, band, use_nugget=True, bin_func=bin_Custom, maxlag = bin_Custom[-1])
            Sill = V_Default.parameters[1] + V_Default.parameters[2]
            list_Sill.append(Sill)
        else:
            list_CV.append(0)
            list_Sill.append(0)
    else:
        list_CV.append(0)
        list_Sill.append(0)
    # Print
    print(f"The {i}th image has been calculated!")
    temp_Check = True

The 0th image has been calculated!
The 1th image has been calculated!
The 2th image has been calculated!
The 3th image has been calculated!
The 4th image has been calculated!
The 5th image has been calculated!
The 6th image has been calculated!
The 7th image has been calculated!
The 8th image has been calculated!
The 9th image has been calculated!
The 10th image has been calculated!
The 11th image has been calculated!
The 12th image has been calculated!
The 13th image has been calculated!
The 14th image has been calculated!
The 15th image has been calculated!
The 16th image has been calculated!
The 17th image has been calculated!
The 18th image has been calculated!
The 19th image has been calculated!
The 20th image has been calculated!
The 21th image has been calculated!
The 22th image has been calculated!
The 23th image has been calculated!
The 24th image has been calculated!
The 25th image has been calculated!
The 26th image has been calculated!
The 27th image has been calculated!
Th

In [27]:
point.getInfo()

{'crs': {'type': 'name', 'properties': {'name': 'EPSG:32634'}},
 'type': 'Point',
 'coordinates': [889895, 90145]}

In [ ]:
# 0th - 87th done
# next 88th!

In [26]:
m = pd.DataFrame({
        "Date": list_Date,
    "CV": list_CV,
    "Sill": list_Sill
})
m.head(5)

,Date,CV,Sill
0,20230121,0.101557,48.347135
1,20230123,0.178041,2.469366
2,20230202,0.107435,46.471934
3,20230302,0.117443,42.922853
4,20230304,-0.208565,0.671741


In [27]:
m.to_csv(cwd + "//GEE " + name_Site + ".csv", index = False)

In [25]:
n = pd.DataFrame({
    "Date": list_Date
})
n

,Date
0,20230121
1,20230123
2,20230202
3,20230302
4,20230304
...,...
62,20231117
63,20231202
64,20231224
65,20231227


In [19]:
n.to_csv(cwd + "//GEE " + name_Site + "Dates.csv", index = False)